In [1]:
import os
%pwd
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\rvaib\\OneDrive\\Desktop\\DogVsCat-Identidier-full-implementation'

In [3]:
# # MLFLOW_TRACKING_URI=https://dagshub.com/Rvaibhavv/DogVsCat-Identidier-full-implementation.mlflow \
# # MLFLOW_TRACKING_USERNAME=Rvaibhavv \
# # MLFLOW_TRACKING_PASSWORD=a37ad20e3c8a402cd86cae1238ec2ef1bed9ba0d \

# # MLFLOW_TRACKING_URI=https://dagshub.com/Rvaibhavv/DogVsCat-Identidier-full-implementation.mlflow \
# # MLFLOW_TRACKING_USERNAME=Rvaibhavv \
# # MLFLOW_TRACKING_PASSWORD=a37ad20e3c8a402cd86cae1238ec2ef1bed9ba0d \
# # python script.py
# # python script.py
# MLFLOW_TRACKING_URI=https://dagshub.com/Rvaibhavv/DogVsCat-Identidier-full-implementation.mlflow \
# MLFLOW_TRACKING_USERNAME=Rvaibhavv \
# MLFLOW_TRACKING_PASSWORD=a37ad20e3c8a402cd86cae1238ec2ef1bed9ba0d \
# python script.py

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Rvaibhavv/DogVsCat-Identidier-full-implementation.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Rvaibhavv"
os.environ["MLFLOW_TRACKING_PASSWORD"]="a37ad20e3c8a402cd86cae1238ec2ef1bed9ba0d"

In [5]:
import tensorflow as tf

In [6]:
model =tf.keras.models.load_model("artifacts/model_trainer/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    batch_size: int
    model_path: Path
    mlflow_uri: str
    all_params: dict
    image_width: int
    image_height: int
    metric_file_name: Path
    model_path: Path
    

In [8]:
from src.DogvsCatClassifier.constants import *
from src.DogvsCatClassifier.utils.common import read_yaml, create_directories,save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:

        config =self.config.model_evaluation
        params =self.params

        create_directories([config.root_dir])
        eval_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            
            mlflow_uri=config.mlflow_uri,
            all_params=self.params,
            test_data_path=config.test_data_path,
            batch_size=params.batch_size,
            image_width=params.image_width,
            image_height=params.image_height,
            metric_file_name=config.metric_file_name,
            model_path= config.model_path
            
            
            

        )
        return eval_config




In [10]:
from tensorflow import keras
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def set_and_test(self):
        validation_ds =keras.utils.image_dataset_from_directory(
            directory = self.config.test_data_path,
            labels='inferred',
            label_mode='int',
            batch_size=self.config.batch_size,
            image_size=(self.config.image_width,self.config.image_height),

        )
        def process(image,label):
            image = tf.cast(image/255. , tf.float32)
            return image,label
        
        
        validation_ds=validation_ds.map(process)

        return  validation_ds
    
    def evaluate(self,validation_ds):
        test_loss, test_accuracy = model.evaluate(validation_ds)
        print('Test accuracy:', test_accuracy)
        print('Test Loss',test_loss)
        score = {"loss": test_loss, "accuracy": test_accuracy}
        save_json(path=Path(self.config.metric_file_name), data=score)
        return test_loss,test_accuracy

    

    def mlflowlogin(self,loss,accuracy):
        
        model =self.config.model_path
        model =tf.keras.models.load_model(model)
        

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run() :
            
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metrics(
                {"loss": loss, "accuracy": accuracy}
            )
            
            
            if tracking_url_type_store != "file":

                
                mlflow.keras.log_model(model, "model", registered_model_name="catvsdogpred")
            else:
                mlflow.keras.log_model(model, "model")
            



In [12]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(model_eval_config)
    validation_ds = model_evaluation.set_and_test()
    loss,accuracy = model_evaluation.evaluate(validation_ds)
    model_evaluation.mlflowlogin(loss,accuracy)


except Exception as e:
    raise e

[2024-01-31 19:10:06,025: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-31 19:10:06,027: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-31 19:10:06,028: INFO: common: created directory at: artifacts]
[2024-01-31 19:10:06,029: INFO: common: created directory at: artifacts\model_evaluation]
Found 5000 files belonging to 2 classes.
125/125 [==============================] - 10s 47ms/step - loss: 0.6504 - accuracy: 0.5992
Test accuracy: 0.5992000102996826
Test Loss 0.6503626704216003
[2024-01-31 19:10:16,523: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}